In [6]:
#importamos las librerias que se van a usar
import pandas as pd

# Leer un archivo Excel y seleccionar una hoja en específico
df = pd.read_excel("DataSet_2.xlsx", sheet_name="Sheet1")

#Se va llenar los vacios de los que si saben ingles
df.loc[
    (df['SABE INGLES (SI/NO)'] == 'Si') & (df['Nivel de Ingles'].isna()),
    'Nivel de Ingles'
] = 'Basico'

#Llenamos los campos vacios
df['Familiares USA'] = df['Familiares USA'].fillna('No indicó')

In [7]:

def calcular_vulnerabilidad(row):
    v = 0
    
    # Edad
    if row["EDAD"] < 25: v += 2
    elif row["EDAD"] < 35: v += 0
    elif row["EDAD"] < 45: v += 0.5
    elif row["EDAD"] < 55: v += 1
    else: v += 2

    # Sexo
    if row["SEXO"] == "Mujer": v += 1

    # Familiares en USA
    if row["Familiares USA"] == "Sí": v -= 1
    else: v += 1

    # Escolaridad
    escolaridad = row["Escolaridad"]
    if escolaridad == "Sin estudios": v += 3
    elif escolaridad == "Primaria": v += 2.5
    elif escolaridad == "Básicos": v += 2
    elif escolaridad == "Diversificado": v += 1.5
    elif escolaridad == "Universitario": v += 1
    elif escolaridad == "Postgrado": v += 0.5
    elif escolaridad == "No indicó": v += 1.5

    # Inglés
    ingles = row["SABE INGLES (SI/NO)"]
    if ingles == "No" : v+=2

    nivel = row["Nivel de Ingles"]
    if nivel == "Basico": v += 1.5
    elif nivel == "Intermedio": v += 1
    elif nivel == "Avanzado": v += 0.5

    # IDHM
    if row["IDHM 2002"] < 0.55: v += 2
    elif row["IDHM 2002"] < 0.65: v += 1.5
    elif row["IDHM 2002"] < 0.75: v += 1
    else: v += 0.5

    # IPM
    if row["IPM"] > 0.3: v += 2
    elif row["IPM"] > 0.2: v += 1.5
    else: v += 1

    # IPM empleo
    if row["IPM EMPLEO"] > 0.3: v += 2
    elif row["IPM EMPLEO"] > 0.2: v += 1.5
    else: v += 0.5

    # Años de experiencia
    exp = row["Año Experiencia"]
    if exp == 0: v += 2
    elif exp <= 5: v += 1.5
    elif exp <= 10: v += 1
    elif exp >10: v += 0.5
    else: v += 1.5

    #Tasa de desempleo por edad
    if row["unemployment_rate"] > 5: v += 2
    elif row["unemployment_rate"] > 3: v += 1.5
    elif row["unemployment_rate"] > 1: v += 1
    else: v += 0.5

    # Clasificación final
    if v <= 7: clasificacion = "Baja"
    elif v <= 12: clasificacion = "Media"
    else: clasificacion = "Alta"

    return pd.Series([v, clasificacion])

# Aplicar función
df[["Score", "Vulnerabilidad"]] = df.apply(calcular_vulnerabilidad, axis=1)

print(df)

       Id SEXO  EDAD   Departamento                    Municipio  \
0    I001    M    34          peten                  desconocido   
1    I002    M    29     chiquimula                  desconocido   
2    I003    M    19      guatemala                  villa nueva   
3    I004    M    19         izabal               puerto barrios   
4    I005    M    29         quiche                  desconocido   
..    ...  ...   ...            ...                          ...   
612  I642    M    21  huehuetenango                       soloma   
613  I643    M    38         quiche                      joyabaj   
614  I644    M    33         quiche                     uspantan   
615  I645    M    41      guatemala                    guatemala   
616  I646    M    37   alta verapaz  fray bartolome de las casas   

       Escolaridad     NIVEL ESCOLARIDAD SABE INGLES (SI/NO) Nivel de Ingles  \
0    Diversificado         Diversificado                  No             NaN   
1     Sin estudios     

In [8]:
#Exportamos el dataset que se utilizar para entrenar el modelo
df.to_excel("Vulnerabilidades.xlsx", index=False)